In [1]:
import pymongo as pm
from pymongo import MongoClient
from bson import Code
# client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017/')
stockDB = client.stock
companyDB = client.companyData
stockCol = stockDB.stocks
companyCol = companyDB.companies

import warnings
warnings.filterwarnings('ignore')

Lookups in MongoDB gives us a way to join with other documents. Lets split our company collection and see the interesting behaviors of Lookup.

In [2]:
companyCol.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'permalink': 'abc3',
 'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
 'homepage_url': 'http://adventnet.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'manageengine',
 'category_code': 'enterprise',
 'number_of_employees': 600,
 'founded_year': 1996,
 'deadpooled_year': 2,
 'tag_list': '',
 'alias_list': 'Zoho ManageEngine ',
 'email_address': 'pr@adventnet.com',
 'phone_number': '925-924-9500',
 'description': 'Server Management Software',
 'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
 'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
 'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
 'image': {'avail

In [5]:
#create a new collection 'company info'

list(companyCol.aggregate([
    {"$project": {"name" :1, "homepage_url" : 1, "founded_year" : 1, "description " : 1, 
                  "number_of_employees " : 1, "overview" : 1, "category_code" : 1}},
    {"$out" : "companyInfo"}
#     {"$limit" : 5}
]))

[]

In [9]:
#create a new collection 'emp info'

list(companyCol.aggregate([
    {"$project": {"name" :1, "relationships" : 1, "arrayCnt" : {"$size" : "$relationships"}}},
    {"$out" : "empInfo"}
#     {"$limit" : 5}
]))

[]

In [16]:
#create a new collection 'emp info1'

list(companyCol.aggregate([
    {"$project": {"_id" : 0, "name" :1, "relationships" : 1, "arrayCnt" : {"$size" : "$relationships"}}},
    {"$unwind": {"path": "$relationships"}},
    {"$out" : "empInfo1"}
#     {"$limit" : 5}
]))

[]

In [17]:
companyDB.collection_names()

['caplog',
 'system.views',
 'empInfo1',
 'companyInfo',
 'companies',
 'empInfo',
 'companyOverview']

In [18]:
compInfo = companyDB.companyInfo
empInfo = companyDB.empInfo
empInfo1 = companyDB.empInfo1

In [13]:
#lets dive in straight away

list(compInfo.aggregate([
   {"$lookup":{
       "from" : "empInfo",
       "localField" : "_id",
       "foreignField" : "_id",
       "as": "empArr"
   }},
    {"$limit" : 5}
]))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'homepage_url': 'http://adventnet.com',
  'category_code': 'enterprise',
  'founded_year': 1996,
  'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
  'empArr': [{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
    'name': 'AdventNet',
    'relationships': [{'is_past': True,
      'title': 'CEO and Co-Founder',
      'person': {'first_name': 'Sridhar',
       'last_name': 'Vembu',
       'permalink': 'sridhar-vembu'}},
     {'is_past': True,
      'title': 'VP of Business Dev',
      'person': {'first_name': 'Neil',
       'last_name': 'Butani',
       'permalink': 'neil-butani'}},
     {'is_past': True,
      'title': 'Usabiliy Engineer',
      'p

In [19]:
empInfo1.find_one()

{'_id': ObjectId('5e4a60d4b5a1efe3c7fd80b9'),
 'name': 'AdventNet',
 'relationships': {'is_past': True,
  'title': 'CEO and Co-Founder',
  'person': {'first_name': 'Sridhar',
   'last_name': 'Vembu',
   'permalink': 'sridhar-vembu'}},
 'arrayCnt': 7}

In [20]:
#lets join again with another collection. This time _id wont macth, so lets use name

list(compInfo.aggregate([
   {"$lookup":{
       "from" : "empInfo1",
       "localField" : "name",
       "foreignField" : "name",
       "as": "empArr"
   }},
    {"$limit" : 5}
]))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'homepage_url': 'http://adventnet.com',
  'category_code': 'enterprise',
  'founded_year': 1996,
  'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
  'empArr': [{'_id': ObjectId('5e4a60d4b5a1efe3c7fd80b9'),
    'name': 'AdventNet',
    'relationships': {'is_past': True,
     'title': 'CEO and Co-Founder',
     'person': {'first_name': 'Sridhar',
      'last_name': 'Vembu',
      'permalink': 'sridhar-vembu'}},
    'arrayCnt': 7},
   {'_id': ObjectId('5e4a60d4b5a1efe3c7fd80ba'),
    'name': 'AdventNet',
    'relationships': {'is_past': True,
     'title': 'VP of Business Dev',
     'person': {'first_name': 'Neil',
      'last_name': 'Butani',
  

As we can see all the matching docs are converted into an array and added to the corresponding doc.

Lookups are always "left join". If there is no match on "from" doc, we'll be having an empty array as result of lookup.

If we want to perform inner join, we can update our aggregator with one more step of "match" that will remove the non matched element (empty array).

In [25]:
#we can either use fields to match or pipelines to match. Lets see an example using pipeline to match.
#here 'compInfo' is input doc and empInfo is 'from' doc. To access the variables inside pipeline, we need to 'declare' variables 
# in the 'let' stage which can be used inside pipeline with '$$'

list(compInfo.aggregate([
   {"$lookup":{
       "from" : "empInfo1",
       "let" : {"cName" : "$name"},
       "pipeline" : [
           {"$match": {"$expr":
           {
            "$eq" : ["$$cName", "$name"]   
           }
           }
           },
           {"$project": {"_id": 0, "name" : 0}}
       ],
       "as": "empArr"
   }},
    {"$limit" : 5}
]))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'homepage_url': 'http://adventnet.com',
  'category_code': 'enterprise',
  'founded_year': 1996,
  'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
  'empArr': [{'relationships': {'is_past': True,
     'title': 'CEO and Co-Founder',
     'person': {'first_name': 'Sridhar',
      'last_name': 'Vembu',
      'permalink': 'sridhar-vembu'}},
    'arrayCnt': 7},
   {'relationships': {'is_past': True,
     'title': 'VP of Business Dev',
     'person': {'first_name': 'Neil',
      'last_name': 'Butani',
      'permalink': 'neil-butani'}},
    'arrayCnt': 7},
   {'relationships': {'is_past': True,
     'title': 'Usabiliy Engineer',
     'person': {'fir

In [26]:
#with pipeline we can more variables and conditions. We're going to join the collections using 'name' and only if we've more than 
# 15 employees ('arrayCnt') in the empInfo.

list(compInfo.aggregate([
   {"$lookup":{
       "from" : "empInfo",
       "let" : {"cName" : "$name"},
       "pipeline" : [
           {"$match": {"$expr": { "$and" : [
           {"$eq" : ["$$cName", "$name"]},
               {"$gt" : ["$arrayCnt", 15]}
           ]
           }
                      }
           },
           {"$project": {"_id": 0, "name" : 0}}
       ],
       "as": "empArr"
   }},
    {"$limit" : 5}
]))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'homepage_url': 'http://adventnet.com',
  'category_code': 'enterprise',
  'founded_year': 1996,
  'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
  'empArr': []},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'homepage_url': 'http://www.omnidrive.com',
  'category_code': 'network_hosting',
  'founded_year': 2005,
  'overview': '<p>Currently in public beta, Omnidrive makes it easy to access, edit and share your files from any computer with a web browser. The site was founded in late 2004 by Nik Cubrilovic and later launched at the November 2006 Web 2.0 Conference.  An interesting slew of features make Omnidrive an att

In [28]:
#There's one more interesting behaviour in lookup. Let's create two collections to test it out

"""
db.classes.insert( [
   { _id: 1, title: "Reading is ...", enrollmentlist: [ "giraffe2", "pandabear", "artie" ], days: ["M", "W", "F"] },
   { _id: 2, title: "But Writing ...", enrollmentlist: [ "giraffe1", "artie" ], days: ["T", "F"] }
])

db.members.insert( [
   { _id: 1, name: "artie", joined: new Date("2016-05-01"), status: "A" },
   { _id: 2, name: "giraffe", joined: new Date("2017-05-01"), status: "D" },
   { _id: 3, name: "giraffe1", joined: new Date("2017-10-01"), status: "A" },
   { _id: 4, name: "panda", joined: new Date("2018-10-11"), status: "A" },
   { _id: 5, name: "pandabear", joined: new Date("2018-12-01"), status: "A" },
   { _id: 6, name: "giraffe2", joined: new Date("2018-12-01"), status: "D" }
])
"""

classCol = stockDB.classes
memberCol = stockDB.members

In [29]:
classCol.find_one()

{'_id': 1.0,
 'title': 'Reading is ...',
 'enrollmentlist': ['giraffe2', 'pandabear', 'artie'],
 'days': ['M', 'W', 'F']}

In [30]:
list(memberCol.find().limit(5))

[{'_id': 1.0,
  'name': 'artie',
  'joined': datetime.datetime(2016, 5, 1, 0, 0),
  'status': 'A'},
 {'_id': 2.0,
  'name': 'giraffe',
  'joined': datetime.datetime(2017, 5, 1, 0, 0),
  'status': 'D'},
 {'_id': 3.0,
  'name': 'giraffe1',
  'joined': datetime.datetime(2017, 10, 1, 0, 0),
  'status': 'A'},
 {'_id': 4.0,
  'name': 'panda',
  'joined': datetime.datetime(2018, 10, 11, 0, 0),
  'status': 'A'},
 {'_id': 5.0,
  'name': 'pandabear',
  'joined': datetime.datetime(2018, 12, 1, 0, 0),
  'status': 'A'}]

In [31]:
#we can match against an array and it'll give us the matching elements for each match in the array

list(classCol.aggregate([
   {
      "$lookup":
         {
            "from": "members",
            "localField": "enrollmentlist",
            "foreignField": "name",
            "as": "enrollee_info"
        }
   }
]))

[{'_id': 1.0,
  'title': 'Reading is ...',
  'enrollmentlist': ['giraffe2', 'pandabear', 'artie'],
  'days': ['M', 'W', 'F'],
  'enrollee_info': [{'_id': 1.0,
    'name': 'artie',
    'joined': datetime.datetime(2016, 5, 1, 0, 0),
    'status': 'A'},
   {'_id': 5.0,
    'name': 'pandabear',
    'joined': datetime.datetime(2018, 12, 1, 0, 0),
    'status': 'A'},
   {'_id': 6.0,
    'name': 'giraffe2',
    'joined': datetime.datetime(2018, 12, 1, 0, 0),
    'status': 'D'}]},
 {'_id': 2.0,
  'title': 'But Writing ...',
  'enrollmentlist': ['giraffe1', 'artie'],
  'days': ['T', 'F'],
  'enrollee_info': [{'_id': 1.0,
    'name': 'artie',
    'joined': datetime.datetime(2016, 5, 1, 0, 0),
    'status': 'A'},
   {'_id': 3.0,
    'name': 'giraffe1',
    'joined': datetime.datetime(2017, 10, 1, 0, 0),
    'status': 'A'}]}]

In [33]:
#we can lookup on objects as well

"""
db.classes_test.insert( [{ _id: 1, title: "Reading is ...", enrollmentlist: [ {"name": "giraffe2"}, {"name": "pandabear"}, {"name": "artie"} ], days: ["M", "W", "F"] },{ _id: 2, title: "But Writing ...", enrollmentlist: [ {"name": "giraffe1"}, {"name": "artie"} ], days: ["T", "F"] }])
"""

ctl = stockDB.classes_test

list(ctl.aggregate([
   {
      "$lookup":
         {
            "from": "members",
            "localField": "enrollmentlist.name",
            "foreignField": "name",
            "as": "enrollee_info"
        }
   }
]))

[{'_id': 1.0,
  'title': 'Reading is ...',
  'enrollmentlist': [{'name': 'giraffe2'},
   {'name': 'pandabear'},
   {'name': 'artie'}],
  'days': ['M', 'W', 'F'],
  'enrollee_info': [{'_id': 1.0,
    'name': 'artie',
    'joined': datetime.datetime(2016, 5, 1, 0, 0),
    'status': 'A'},
   {'_id': 5.0,
    'name': 'pandabear',
    'joined': datetime.datetime(2018, 12, 1, 0, 0),
    'status': 'A'},
   {'_id': 6.0,
    'name': 'giraffe2',
    'joined': datetime.datetime(2018, 12, 1, 0, 0),
    'status': 'D'}]},
 {'_id': 2.0,
  'title': 'But Writing ...',
  'enrollmentlist': [{'name': 'giraffe1'}, {'name': 'artie'}],
  'days': ['T', 'F'],
  'enrollee_info': [{'_id': 1.0,
    'name': 'artie',
    'joined': datetime.datetime(2016, 5, 1, 0, 0),
    'status': 'A'},
   {'_id': 3.0,
    'name': 'giraffe1',
    'joined': datetime.datetime(2017, 10, 1, 0, 0),
    'status': 'A'}]}]